### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
importlib.reload(train)

models = {
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50, 70, 80, 200],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 50, 70, 100],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7]
    }],
    "XGB": [{
        "use_label_encoder": [False],
        "objective": ["binary:logistic"],
        "eval_metric": ["error", "mean_absolute_error", "mean_squared_error", "log_loss"]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [5]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.2, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    # features = {name: metrics for name in models.keys()}
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=4)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

One or more of the test scores are non-finite: [0.60277976 0.6041207  0.61632223 0.62432926 0.61664142 0.61942874
 0.56912175 0.60517601 0.61306251 0.62143093 0.6326698  0.63176902
 0.51888695 0.57815733 0.59757099 0.61197012 0.61757299 0.62700877
 0.57817657 0.5958038  0.59603263 0.60447012 0.60685386 0.61004312
 0.59562917 0.6134173  0.58582674 0.59995139 0.61952991 0.62455345
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.61692306        nan        nan        nan]


LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 70}
XGB: {'eval_metric': 'error', 'objective': 'binary:logistic', 'use_label_encoder': False}
Model LGB, Actual: [0 1], [111 114], Predictions: [0 1], [135  90]
Model RF, Actual: [0 1], [111 114], Predictions: [0 1], [125 100]
Model XGB, Actual: [0 1], [111 114], Predictions: [0 1], [149  76]

[('hf_rr', 224), ('lf_rr', 210), ('rmssd', 206), ('bpm', 174)]


[('rmssd', 0.2936771117626445), ('bpm', 0.2563499253585739), ('lf_rr', 0.22949072083150343), ('hf_rr', 0.22048224204727823)]


[('bpm', 0.2851473), ('hf_rr', 0.25624493), ('rmssd', 0.23518357), ('lf_rr', 0.2234242)]



One or more of the test scores are non-finite: [0.58179426 0.62316955 0.63225722 0.64131767 0.62997349 0.62659064
 0.63005903 0.63692618 0.61947536 0.63219056 0.63069866 0.62358633
 0.61909236 0.65961206 0.64415398 0.65374199 0.63689823 0.63847481
 0.61893158 0.6434586  0.64112968 0.64134317 0.63249119 0.62494946
 0.60369778 0.61937785 0.61367839 0.61671009 0.60844298 0.6185491
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.61513538        nan        nan        nan]


LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 20}
XGB: {'eval_metric': 'error', 'objective': 'binary:logistic', 'use_label_encoder': False}
Model LGB, Actual: [0 1], [112 111], Predictions: [0 1], [135  88]
Model RF, Actual: [0 1], [112 111], Predictions: [0 1], [137  86]
Model XGB, Actual: [0 1], [112 111], Predictions: [0 1], [157  66]

[('hf_rr', 439), ('bpm', 419), ('lf_rr', 389), ('rmssd', 351)]


[('lf_rr', 0.28321863529819524), ('hf_rr', 0.25057990308619515), ('rmssd', 0.24395123482049402), ('bpm', 0.22225022679511555)]


[('lf_rr', 0.2817231), ('bpm', 0.25734845), ('rmssd', 0.25047836), ('hf_rr', 0.21045013)]



One or more of the test scores are non-finite: [0.56419407 0.61347433 0.64339366 0.66913608 0.68120504 0.68892669
 0.62429638 0.6471312  0.64710893 0.66639592 0.67732438 0.68277948
 0.61639242 0.67329632 0.6666831  0.67640363 0.68520365 0.69029976
 0.60850798 0.67127059 0.67461003 0.68281799 0.70002366 0.69745454
 0.62429477 0.64446725 0.66909729 0.67884986 0.68831212 0.68616797
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.64473149        nan        nan        nan]


LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 70}
XGB: {'eval_metric': 'error', 'objective': 'binary:logistic', 'use_label_encoder': False}
Model LGB, Actual: [0 1], [94 85], Predictions: [0 1], [127  52]
Model RF, Actual: [0 1], [94 85], Predictions: [0 1], [114  65]
Model XGB, Actual: [0 1], [94 85], Predictions: [0 1], [102  77]

[('bpm', 229), ('rmssd', 227), ('hf_rr', 215), ('lf_rr', 175)]


[('rmssd', 0.2703471534978716), ('bpm', 0.2574493906956561), ('lf_rr', 0.24237960334735975), ('hf_rr', 0.22982385245911266)]


[('rmssd', 0.28142604), ('hf_rr', 0.276341), ('bpm', 0.25282317), ('lf_rr', 0.18940987)]



One or more of the test scores are non-finite: [0.61521918 0.61066554 0.59741198 0.61031523 0.61106045 0.6333598
 0.60840856 0.5989574  0.59705889 0.5987344  0.59584727 0.60171367
 0.58994249 0.60472064 0.5936025  0.59803454 0.59978473 0.6132946
 0.58855205 0.57552664 0.58232287 0.60040318 0.60054893 0.61597262
 0.58126009 0.58694982 0.58090855 0.58772438 0.58590301 0.60024731
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.61900672        nan        nan        nan]


LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 100}
XGB: {'eval_metric': 'error', 'objective': 'binary:logistic', 'use_label_encoder': False}
Model LGB, Actual: [0 1], [118  84], Predictions: [0 1], [162  40]
Model RF, Actual: [0 1], [118  84], Predictions: [0 1], [154  48]
Model XGB, Actual: [0 1], [118  84], Predictions: [0 1], [138  64]

[('bpm', 399), ('rmssd', 353), ('hf_rr', 298), ('lf_rr', 258)]


[('bpm', 0.3330015464940695), ('lf_rr', 0.2373363729514384), ('hf_rr', 0.21835855609539684), ('rmssd', 0.2113035244590953)]


[('hf_rr', 0.29039967), ('lf_rr', 0.27446422), ('bpm', 0.23537219), ('rmssd', 0.19976388)]



One or more of the test scores are non-finite: [0.56022199 0.52965957 0.54875502 0.53522081 0.54976785 0.55778556
 0.57192522 0.52172082 0.53701083 0.54596861 0.55028025 0.53418145
 0.59737716 0.56096987 0.5667471  0.53776063 0.53935471 0.52904269
 0.54414713 0.52933184 0.52865169 0.52247666 0.52280417 0.52005149
 0.53017648 0.51001612 0.51847586 0.53067633 0.546904   0.53935308
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
One or more of the test scores are non-finite: [0.57249953        nan        nan        nan]


LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 10}
XGB: {'eval_metric': 'error', 'objective': 'binary:logistic', 'use_label_encoder': False}
Model LGB, Actual: [0 1], [140  61], Predictions: [0 1], [133  68]
Model RF, Actual: [0 1], [140  61], Predictions: [0 1], [127  74]
Model XGB, Actual: [0 1], [140  61], Predictions: [0 1], [121  80]

[('bpm', 375), ('hf_rr', 354), ('rmssd', 341), ('lf_rr', 321)]


[('hf_rr', 0.29005088399191736), ('bpm', 0.2533397945560006), ('lf_rr', 0.2379164917049505), ('rmssd', 0.21869282974713156)]


[('rmssd', 0.262846), ('bpm', 0.259759), ('lf_rr', 0.25734034), ('hf_rr', 0.22005466)]

Model evaluation metrics for LGB:
	Accuracy: 0.49333333333333335
	Precision: 0.5
	Recall: 0.39473684210526316
	F1-score: 0.4411764705882353
	AUC score: 0.49466571834992884
----------------------------------------
	Accuracy: 0.5201793721973094
	Precision: 0.522

In [ ]:
# LOAD TRAIN AND TEST DATA -- train on some phases and test on others
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

# metrics = train.Metrics.ALL
# metrics = train.Metrics.ECG \
    # + train.Metrics.EDA \
    # + train.Metrics.ANKLE + train.Metrics.WRIST

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    # train.Metrics.HF_RR, 
    # train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]

phases_a = [
    "Baseline_Rest", 
    "BugBox_Relax", 
    "BugBox_Anticipate", 
    # "BugBox_Exposure", 
    # "BugBox_Break",
    "Speech_Relax", 
    "Speech_Anticipate", 
    # "Speech_Exposure", 
    # "Speech_Break"
]

phases_b = [
    # "Baseline_Rest", 
    # "BugBox_Relax", 
    # "BugBox_Anticipate", 
    "BugBox_Exposure", 
    # "BugBox_Break",
    # "Speech_Relax", 
    # "Speech_Anticipate", 
    "Speech_Exposure", 
    # "Speech_Break"
]

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

anxiety_label_type = "Anxiety"
# anxiety_label_type = None
test_size = 1.0

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, phases_a, verbose=False, anxiety_label_type=anxiety_label_type)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_b, verbose=False, anxiety_label_type=anxiety_label_type)
x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# drop subjects with noisy data

x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)]
y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)]
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)]
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)]

print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

# x = x[x['subject'] != 8.0]
# y = y[y['subject'] != 8.0]

if anxiety_label_type is not None:
    x_a.drop(labels=["anxietyGroup"], axis=1)
    x_b.drop(labels=["anxietyGroup"], axis=1)

# 0-1 scaling
for i in range(3, len(x_a.columns)):
    data_col = x_a[x_a.columns[i]]
    data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    x_a[x_a.columns[i]] = data_col
for i in range(3, len(x_b.columns)):
    data_col = x_b[x_b.columns[i]]
    data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    x_b[x_b.columns[i]] = data_col
    
# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
reports = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=False, save_metrics=True, test_size=test_size)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.4:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("")
print("\n")